In [5]:
import os
import json
import re

In [27]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = text.strip()
    return text

def load_data(data_filepath, split_filepath):
    
    train_data, test_data = [], []

    with open(split_filepath, 'r') as file:
        splits = json.load(file)
        train_ids = splits['train']
        test_ids = splits['test']
        
    with open(filepath, 'r') as file:
        for line in file:
            item = json.loads(line)
            kept_annotations = [item[key] for key in item.keys() if key.startswith("Answer.Q1_")]
            if len(kept_annotations) == 0:
                continue
            texts = [
                clean_text(item['context8_tweettext']),
                clean_text(item['context9_tweettext']),
                clean_text(item['context10_tweettext']),
                clean_text(item['context11_tweettext']),
                clean_text(item['context12_tweettext']),
                clean_text(item['context13_tweettext']),
            ]
            instance = {'texts': texts, 'label': item['adjudicated_label'], 'location': item['anchor_location']}
            if item['instance_id'] in train_ids:
                train_data.append(instance)
            if item['instance_id'] in test_ids:
                test_data.append(instance)
                
    return train_data, test_data


data_filepath = 'data/data.json'
split_filepath = 'data/data_split'
train_data, test_data = load_data(data_filepath, split_filepath)

In [1]:
# import os

# os.environ["CUDA_VISIBLE_DEVICES"]="3"

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = 'google/flan-ul2'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, load_in_8bit=True).to('cuda:2')
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 23.63 GiB total capacity; 22.38 GiB already allocated; 75.56 MiB free; 22.56 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model_input = """### Prompt: Read the tweets below and determine its sentiment.
### Tweets: Dallas is so bad.
OPTIONS:
1. Negative
2. Positive
### Answer: """

inputs = tokenizer(model_input, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))